In [1]:
import pandas as pd
from Twitterpreprocessor import TwitterPreprocessor
import matplotlib.pyplot as plt
import tweepy
from decouple import config
import json
import requests


In [74]:

import joblib
import spacy
nlp = spacy.load('en_core_web_sm')
consumer_key = config('TWITTER_CONSUMER_KEY')
consumer_secret = config('TWITTER_CONSUMER_SECRET')
tiktok_key=config('TIKTOK_KEY')

def get_twitter_poste(id_poste,access_token,access_token_secret) -> str:
        
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        api = tweepy.API(auth, wait_on_rate_limit=True)
        tweet = api.get_status(id_poste)
        tweet = tweet._json
        return tweet['text']

def get_tiktok_poste(id_poste,refrech_token) -> str:
        getUrl = f"https://open-api.tiktok.com/oauth/refresh_token?client_key={tiktok_key}&grant_type=refresh_token&refresh_token={refrech_token}"
        resMedia = requests.get(getUrl)
        accessToken = json.loads(resMedia.text)['data']['access_token']
        api_endpoint = "https://open-api.tiktok.com/video/query/"
        # Set your payload (data)
        payload = {
            "access_token":accessToken,       
            "filters": {
                "video_ids": [
                   id_poste
                ]
            },
        'fields': ["duration","title","video_description"],
        }

        response = requests.post(api_endpoint, json=payload)
        v = response.json()['data']['videos'][0]
        return v["video_description"]+" "+v["title"]
from nltk.stem import PorterStemmer
def porterstemmer(text):
  ps = PorterStemmer()
  text = ' '.join(ps.stem(word) for word in text.split() if word in text)
  return text  
def lemmatization (text):
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return ' '.join(tokens)
def get_twitter_inpute_data(id_poste,access_token,access_token_secret):
    text=get_twitter_poste(id_poste,access_token,access_token_secret)
    text=TwitterPreprocessor(str(text)).fully_preprocess().text
    text=porterstemmer(text)
    text=lemmatization(text)
    text =[text]
    vectorizer = joblib.load('vectorizer.pkl')
    text = vectorizer.transform(text)
    return text

def get_tiktok_inpute_data(id_poste,refrech_token):
    text=get_tiktok_poste(id_poste,refrech_token)
    text=TwitterPreprocessor(str(text)).fully_preprocess().text
    text=porterstemmer(text)
    text=lemmatization(text)
    text =[text]
    vectorizer = joblib.load('vectorizer.pkl')
    text = vectorizer.transform(text)
    return text

In [75]:

print(get_tiktok_inpute_data("7112372202607086853","rft.246dfc618e0de9e8a8997ee0d2792bbdSaNq8zdBFdPdX1C7NoHKydWXtoyA!4923"))

{'data': {'videos': [{'video_description': '', 'duration': 8, 'title': ''}], 'cursor': 0, 'has_more': False}, 'error': {'code': 0, 'message': ''}}



In [65]:
import requests
refrech_token="rft.246dfc618e0de9e8a8997ee0d2792bbdSaNq8zdBFdPdX1C7NoHKydWXtoyA!4923"
getUrl = f"https://open-api.tiktok.com/oauth/refresh_token?client_key={tiktok_key}&grant_type=refresh_token&refresh_token={refrech_token}"
resMedia = requests.get(getUrl)
print(json.loads(resMedia.text)['data'])
accessToken = json.loads(resMedia.text)['data']['access_token']
api_endpoint = "https://open-api.tiktok.com/video/query/"


# Set your headers
headers = {
    "Authorization": "Bearer " + accessToken
}

# Set your payload (data)
payload = {
    "access_token":accessToken,
            
    "filters": {
        "video_ids": [
            "7112372202607086853"
        
        ]
    },
  'fields': ["duration","title","video_description"],
}

response = requests.post(api_endpoint, json=payload)
videoInfoResponse = response.json()
print(videoInfoResponse)


{'access_token': 'act.9888fad998875f6827a9c63b22ec1b61pggefqTRm2Yg1q20dNbeZMUmJkh8!4910', 'captcha': '', 'desc_url': '', 'description': '', 'error_code': 0, 'expires_in': 86400, 'log_id': '202305221510239616FD7C0BDB887B41A8', 'open_id': '81d861ee-27e3-4204-82bd-054ae3500f43', 'refresh_expires_in': 2745628, 'refresh_token': 'rft.246dfc618e0de9e8a8997ee0d2792bbdSaNq8zdBFdPdX1C7NoHKydWXtoyA!4923', 'scope': 'user.info.basic,video.list'}
{'data': {'videos': [{'video_description': '', 'duration': 8, 'title': ''}], 'cursor': 0, 'has_more': False}, 'error': {'code': 0, 'message': ''}}
